In [1]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [2]:
dire = "./cleaned-data" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 426, 1: 426}


In [8]:
# load pretrained model
vgg16 = torchvision.models.vgg16(pretrained=True)
googlenet = torchvision.models.googlenet(pretrained=True)
efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [15]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(self.network.fc.in_features, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, 2),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [5]:
# define model to replace here
# model_details = {
#     'model': resnet50,
#     'name': 'resnet50'
# }
# model_details = {
#     'model': efficientnet_b7,
#     'name': 'efficientnet_b7'
# }
model_details = {
    'model': googlenet,
    'name': 'googlenet'
}
# model_details = {
#     'model': vgg16,
#     'name': 'vgg16'
# }

In [12]:
resnet50.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [11]:
googlenet.fc

Linear(in_features=1024, out_features=1000, bias=True)

In [16]:
# define params here
model_name = model_details['name']
net = ClassifierNet(model_details['model'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 50

training.run(model_name, net, optimizer, scheduler, criterion, epochs, trainloader, testloader)

AttributeError: 'Sequential' object has no attribute 'in_features'